In [ ]:
import cv2
import numpy as np
import imutils

video = 'Video.mp4'

cap = cv2.VideoCapture(video)
cnt = 0

if (cap.isOpened() == False):
    print("Error opening video stream or file")

ret, first_frame = cap.read()

while (cap.isOpened()):

    ret, frame = cap.read()

    if ret == True:

        roi = frame[:800,:]
    
        thresh=600
        end = roi.shape[1] - thresh
        roi = roi[:,thresh:end]

        cv2.imshow("Video", roi)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

        cv2.imwrite('frames/'+str(cnt)+'.png', roi)
        cnt = cnt+1

    else:
        break

cv2.destroyAllWindows()

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
import re

frames = os.listdir('frames/')
frames.sort(key=lambda f: int(re.sub('\D', '', f)))

images = []
for i in frames:
    img = cv2.imread('frames/'+i)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.GaussianBlur(img, (25, 25), 0)
    images.append(img)

images = np.array(images)

In [ ]:
nonzero = []
for i in range((len(images)-1)):

    mask = cv2.absdiff(images[i], images[i+1])
    _, mask = cv2.threshold(mask, 50, 255, cv2.THRESH_BINARY)
    num = np.count_nonzero((mask.ravel()))
    nonzero.append(num)

x = np.arange(0, len(images)-1)
y = nonzero

plt.figure(figsize=(20, 4))
plt.scatter(x, y)

In [ ]:
threshold = 60 * 10e3
for i in range(len(images)-1):
    if (nonzero[i] > threshold):
        scene_change_idx = i
        break

frames = frames[:(i+1)]
frames

In [ ]:
img = cv2.imread('frames/' + frames[50])
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (25, 25), 0)

plt.figure(figsize=(5, 10))
plt.imshow(gray, cmap='gray')

In [ ]:
_, mask = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

plt.figure(figsize=(5, 5))
plt.imshow(mask, cmap='gray')

In [ ]:
contours, images = cv2.findContours(
    mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
img_copy = np.copy(gray)
cv2.drawContours(img_copy, contours, -1, (0,255,0), 3)

plt.imshow(img_copy, cmap='gray')

In [ ]:
num = 20
cnt = 0
for i in range(len(contours)):
    x, y, w, h = cv2.boundingRect(contours[i])

    numer = min([w, h])
    denom = max([w, h])
    ratio = numer/denom

    if (x >= num and y >= num):
        xmin, ymin = x-num, y-num
        xmax, ymax = x+w+num, y+h+num
    else:
        xmin, ymin = x, y
        xmax, ymax = x+w, y+h

    if (ratio >= 0.5 and ((w <= 10) and (h <= 10))):
        print(cnt, x, y, w, h)
        cv2.imwrite("patch/"+str(cnt)+".png", img[ymin:ymax, xmin:xmax])
        cnt = cnt+1

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
folders=os.listdir('data/')

images=[]
labels= []
for folder in folders:
    files=os.listdir('data/'+folder)
    for file in files:
        img=cv2.imread('data/'+folder+'/'+file, 0)
        img=cv2.resize(img,(25,25))
        
        images.append(img)
        labels.append(int(folder))

images = np.array(images)
features = images.reshape(len(images),-1)